<a href="https://colab.research.google.com/github/myszojelen23/Emotive-word-matcher-and-calculator/blob/main/Calculating_mean_emotion_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

Importing the Excel sheet with arguments

In [ ]:
files.upload()

In [ ]:
df = pd.read_excel('Argumentacje.xlsx')

Cleaning

In [ ]:
df.rename(columns = {'Imię i nazwisko':'Author', 'Źródło':'Source', 'Typ argumentu (szeregowy lub równoległy)': 'Arg_type', 'Konkluzja':'Conclusion', 'Przesłanka 1':'Premise1', 'Przesłanka 2': 'Premise2', 'Przesłanka 3': 'Premise3', 'Przesłanka 4':'Premise4', 'Przesłanka 5':'Premise5', 'Przesłanka 6':'Premise6', 'Przesłanka 7':'Premise7'}, inplace = True) #renaming columns

df.fillna('', inplace=True)  # Replacing NaN values with string

Joining the text of premises and conclusions in one string



In [ ]:
df['Premises'] = df[['Premise1', 'Premise2', 'Premise3', 'Premise4', 'Premise5', 'Premise6', 'Premise7']].agg(' '.join, axis=1)
df['Premises']

df['Arguments'] = df[['Premises','Conclusion']].agg(' '.join, axis=1)

Uploading NAWL database

In [ ]:
files.upload()

Cleaning NAWL database

In [ ]:
nawl = pd.read_excel('nawl_emo_cat.xlsx')

nawl = nawl.drop(columns=['BAWL_word', 'BE_N_all', 'hap_SD_all', 'ang_SD_all', 'sad_SD_all', 'fea_SD_all', 'dis_SD_all', 'BE_N_women', 'hap_M_women',
       'hap_SD_women', 'ang_M_women', 'ang_SD_women', 'sad_M_women',
       'sad_SD_women', 'fea_M_women', 'fea_SD_women', 'dis_M_women',
       'dis_SD_women', 'BE_N_men', 'hap_M_men', 'hap_SD_men', 'ang_M_men',
       'ang_SD_men', 'sad_M_men', 'sad_SD_men', 'fea_M_men', 'fea_SD_men',
       'dis_M_men', 'dis_SD_men', 'DENN_BAWL', 'Briesemeister_liberal',
       'Briesemeister_conservative', 'Mikels', 'Mikels_blended', 'ED_class'])

nawl = nawl.rename(columns = {'No.':'nr', 'NAWL_word':'word', 'hap_M_all':'happiness', 'ang_M_all':'anger', 'sad_M_all':'sadness', 'fea_M_all':'fear', 'dis_M_all':'disgust'})

Lemmatization

In [ ]:
!pip install --upgrade spacy

In [ ]:
!python -m spacy download pl_core_news_sm

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("pl_core_news_sm")

In [ ]:
text = df.loc[0,'Arguments'] #this line allows to choose one argument (understood as a string created from the premises and the conclusion) from the database
text_all = df['Arguments'].values.tolist()
text_all = ' '.join(text_all)
print(type(text_all))
#the following line allows to just paste a choosen string
# text = 'Światowa Organizacja Zdrowia (WHO) alarmuje, że osoby palące są bardziej narażone na komplikacje związane zarażeniem koronawirusem SARS-CoV-2. Koronawirus atakuje drogi oddechowe i powoduje ich uszkodzenie. Dym tytoniowy ma negatywny wpływ na stan układu oddechowego. U palaczy mogą wystąpić cięższe dolegliwości związane z zarażeniem koronawirusem. Zrezygnuj z używek.'
doc = nlp(text) #text preprocessing

print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

lemma_list = [] #a list of lemmas contained in each argument - empty so far
for token in doc: #each token contained in the argument...
  lemma_list.append(token.lemma_) #is assigned its dictionary form, lemma

lemma_list

In [ ]:
emo_word = nawl['word'] #A segment of the nawl dataset containing a column with words
emo_word_list = emo_word.tolist() #converted into a list

found_words = [] #empty list for words found

def matcher(argument_list, emo_word_list): #function that matches the words found in the arguments to words in the list of emotive words from the NAWL database; if the word is in the database, it goes into the list
  for word in argument_list:
    if word in emo_word_list:
      found_words.append(word)

matcher(lemma_list, emo_word_list)

found_words


In [ ]:
nawl_fragment = nawl[nawl['word'].isin(found_words)] #A part of the NAWL database that contains only the words included in the given argument
mean_values = nawl_fragment[["happiness", "anger", "sadness", "fear", "disgust"]].mean()
mean_values